In [1]:
pip install --upgrade google-cloud-speech

In [2]:
import os

In [3]:
from google.cloud import speech

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='cloud_admin.json'

In [5]:
speech_client = speech.SpeechClient()

In [6]:
# Trancsribing a local media file

In [7]:
# Load the media files (transcribe)

In [8]:
media_file_mp3 = 'speech_short.mp3'
#media_file_wav = 'speech.wav'

In [16]:
with open(media_file_mp3, 'rb') as f1:
    byte_data_mp3 = f1.read()
audio_mp3 = speech.RecognitionAudio(content=byte_data_mp3)

In [17]:
# Configure media file output

In [18]:
config_mp3 = speech.RecognitionConfig(
    sample_rate_hertz = 48000,
    enable_automatic_punctuation = True,
    language_code = 'en-US'
)

In [19]:
# Transcribe

In [20]:
response_standard_mp3 = speech_client.recognize(
    config = config_mp3,
    audio= audio_mp3
)

In [21]:
response_standard_mp3.total_billed_time

datetime.timedelta(seconds=57)

In [22]:
response_standard_mp3

results {
  alternatives {
    transcript: "The biggest poison in his regret."
    confidence: 0.832085311
  }
  result_end_time {
    seconds: 5
    nanos: 640000000
  }
  language_code: "en-us"
}
results {
  alternatives {
    transcript: " But I think there are a lot of people that have dreams and aspirations of things that they always wanted to do, but it wasn\'t the right time. Or they didn\'t have enough money or didn\'t have enough experience. It\'s never the right time. You never going to have the right experience and all the sudden you wake up and you\'re 70."
    confidence: 0.941898108
  }
  result_end_time {
    seconds: 27
    nanos: 880000000
  }
  language_code: "en-us"
}
results {
  alternatives {
    transcript: " And you like. I wish I could have done it."
    confidence: 0.941285908
  }
  result_end_time {
    seconds: 31
    nanos: 460000000
  }
  language_code: "en-us"
}
results {
  alternatives {
    transcript: " It\'s what you don\'t do that screws with you late

In [57]:
media_uri = "gs://bucket_soham_1/speech.mp3"

In [58]:
long_audio_wav = speech.RecognitionAudio(uri=media_uri)

In [ ]:
speech.RecognitionConfig()

In [63]:
config_mp3_enhanced = speech.RecognitionConfig(
    sample_rate_hertz=48000,
    enable_automatic_punctuation=True,
    language_code ='en-US',
    use_enhanced=True,
    model='command_and_search'
)

In [64]:
operation = speech_client.long_running_recognize(
    config = config_mp3_enhanced,
    audio= long_audio_wav
)

In [65]:
print(operation)

In [66]:
response= operation.result()

In [67]:
type(response)


google.cloud.speech_v1.types.cloud_speech.LongRunningRecognizeResponse

In [68]:
for result in response.results:
    print(result.alternatives[0].transcript)
    print(result.alternatives[0].confidence)

Sometimes you need to get knocked down before, you can really figure out what you up with your fight. Is that how you need to fight it?
0.9210127592086792
 Sometimes you need to feel the pain and sting of defeat to activate the real passion and purpose that God predestined inside of you.
0.978005588054657
 God says it's Jeremiah. I know the plans I have for you plans to prosper you and not to harm. You plans to give you hope and a future.
0.9665605425834656
 Kimmy. Well, on this day this day when you have reached the hilltop and you are deciding on our next jobs. Next steps careers further education, you would rather find purpose
0.9218074679374695
 Then a job or career.
0.8985251188278198
 Purpose crosses disciplines.
0.9438664317131042
 Purpose is an essential element of you. It is the reason you're on the planet at this particular time in history, your very existence is wrapped up in the things you are here to fulfill whatever you choose for career path. Remember the struggles along

In [73]:
pip install PyAudio

In [11]:
from __future__ import division

import re
import sys
import ast # we import this for string evaluation

from google.cloud import speech

import pyaudio
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms


class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)


def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = " " * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            
            ## Custom code:
            # We need to now look for keywords such as find, if triggered we need to refer back to the algorithm which returns us stuff
            # look for find/where + keyword of watch/glasses 
            
        
            # FOR DETECTING FOR A WATCH
            if re.search(r"(where).*(watch).*|(find).*(watch).*",transcript,re.I):
                # we write a function here to run the detect.py file
                
                output= !python detect.py --weights best.pt --img 640 --conf 0.1 --source test
                
                print(output)
                
                
                if re.search(r"Watch",output[-1]):
                    
                    output_dict = ast.literal_eval(output[-2])
                    
                    location_of_object = output_dict['Watch']
                    text=f"I found the watch. It is on the {location_of_object}"
                    text_to_speech(text)
                    IPython.display.Audio("Audio.mp3")
                    
                else:
                    text = "object not found"
                       
                    text_to_speech(text)
                    IPython.display.Audio("Audio.mp3")
                    
            
            # FOR DETECTING FOR GLASSES        
            if re.search(r"(where).*(glasses).*|(find).*(glasses).*",transcript,re.I):
                # we write a function here to run the detect.py file
                
                output= !python detect.py --weights best.pt --img 640 --conf 0.1 --source test
            
                
                print(output)
                
                
                if re.search(r"Glasses",output[-1]):
                    
                    output_dict = ast.literal_eval(output[-2])
                    
                    #maybe add a try except block over here
                    location_of_object = output_dict['Glasses']
                    text=f"I found the glasses. It is on the {location_of_object}"
                    text_to_speech(text)
                    IPython.display.Audio("Audio.mp3")

                else:
                    text = "object not found"
                       
                    text_to_speech(text)
                    IPython.display.Audio("Audio.mp3")
                    
                    
            # FOR DETECTIN FOR THE PHONE
            if re.search(r"(where).*(phone).*|(find).*(phone).*",transcript,re.I):
                
                
                output= !python detect.py --weights best.pt --img 640 --conf 0.1 --source test
            
                
                print(output)
                
                
                if re.search(r"Mobile phone",output[-1]):
                    
                    output_dict = ast.literal_eval(output[-2])
                    
                    location_of_object = output_dict['Mobile phone']
                    text=f"I found the phone. It is on the {location_of_object}"
                    text_to_speech(text)
                    IPython.display.Audio("Audio.mp3")

                else:
                    text = "object not found"
                       
                    text_to_speech(text)
                    IPython.display.Audio("Audio.mp3")
                    
                    

                
                    
                #if the object is in the item_list then we return the object with the maximum IOU
                
                #if the object is not in the list we return an audio file saying that we could not find the object
            
            
            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                break

            num_chars_printed = 0


def main():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "en-US"  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
        model ='command_and_search'
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)
        
        
        


if __name__ == "__main__":
    main()

find my watch
["\x1b\x1bdetect: \x1bweights=['best.pt'], source=test, data=data\\coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1", '\x1b\x1brequirements:\x1b YOLOv5 requirements "lxml" "tensorflow==2.3.0rc0" "opencv-python==4.1.1.26" not found, attempting AutoUpdate...', 'ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0rc0 (from versions: 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)', 'ERROR: No matching distribution found for tensorflow==2.3.0rc0', '\x1b\x1brequireme

In [30]:
txt = " watch "

print( re.search("\bwatch\b",txt,re.I))

None


In [18]:
pp= !python detect.py --weights best.pt --img 640 --conf 0.2 --source test
                
print(pp)
                

["\x1b\x1bdetect: \x1bweights=['best.pt'], source=test, data=data\\coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1", 'YOLOv5  2022-11-2 Python-3.10.0 torch-1.13.0+cpu CPU', '', 'Fusing layers... ', 'Model summary: 157 layers, 1764577 parameters, 0 gradients, 4.1 GFLOPs', 'image 1/1 C:\\Users\\soham\\Desktop\\MU CV project\\yolov5\\test\\img1.jpg: 640x448 1 Watch, 181.8ms', 'Speed: 0.0ms pre-process, 181.8ms inference, 0.0ms NMS per image at shape (1, 3, 640, 640)', 'Results saved to \x1bruns\\detect\\exp28\x1b', '20.0', '58.0', '298.0', '408.0', "[{'x1': 20.0, 'x2': 298.0, 'y1': 58.0, 'y2': 408.0}]", 'Watch 0.71', "['Watch']"]


In [19]:
pp[-1]

"['Watch']"

## Text To Speech (TTS)

In [9]:
pip install google-cloud-texttospeech

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from unicodedata import name
from urllib import response
from google.cloud import texttospeech_v1
import IPython

client = texttospeech_v1.TextToSpeechClient()

text = '''<speak> Hi. Your phone is on the ppeed</speak>'''

# Lets functionalize this

def text_to_speech(text):

    synthesis_input = texttospeech_v1.SynthesisInput(ssml=text)

    voice1 = texttospeech_v1.VoiceSelectionParams(
        language_code = 'en-in',
        ssml_gender = texttospeech_v1.SsmlVoiceGender.FEMALE
    )

    print(client.list_voices)
    audio_config = texttospeech_v1.AudioConfig(
        audio_encoding = texttospeech_v1.AudioEncoding.MP3
    )

    response1 = client.synthesize_speech(
            input = synthesis_input,
            voice = voice1,
            audio_config =audio_config
    )

    with open('Audio.mp3','wb',) as output:
        output.write(response1.audio_content)

    
text_to_speech(text)


IPython.display.Audio("Audio.mp3")

<bound method TextToSpeechClient.list_voices of <google.cloud.texttospeech_v1.services.text_to_speech.client.TextToSpeechClient object at 0x00000201BC8F68B0>>


In [18]:
import speech_recognition as sr

ModuleNotFoundError: No module named 'speech_recognition'

In [19]:
import detect


In [21]:
otput = detect.run()

YOLOv5  2022-11-2 Python-3.9.7 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 1764577 parameters, 0 gradients, 4.1 GFLOPs
image 1/1 C:\Users\soham\Desktop\MU CV project\yolov5\test\img1.jpg: 416x640 3 Watchs, 1 Bed, 151.5ms
Speed: 1.0ms pre-process, 151.5ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp201


335.0
228.0
400.0
297.0
[{'x1': 335.0, 'x2': 400.0, 'y1': 228.0, 'y2': 297.0, 'name': 'Watch'}]
Watch 0.13
256.0
219.0
391.0
289.0
[{'x1': 335.0, 'x2': 400.0, 'y1': 228.0, 'y2': 297.0, 'name': 'Watch'}, {'x1': 256.0, 'x2': 391.0, 'y1': 219.0, 'y2': 289.0, 'name': 'Watch'}]
Watch 0.18
0.0
0.0
624.0
329.0
[{'x1': 335.0, 'x2': 400.0, 'y1': 228.0, 'y2': 297.0, 'name': 'Watch'}, {'x1': 256.0, 'x2': 391.0, 'y1': 219.0, 'y2': 289.0, 'name': 'Watch'}, {'x1': 0.0, 'x2': 624.0, 'y1': 0.0, 'y2': 329.0, 'name': 'Bed'}]
Bed 0.39
285.0
214.0
330.0
284.0
[{'x1': 335.0, 'x2': 400.0, 'y1': 228.0, 'y2': 297.0, 'name': 'Watch'}, {'x1': 256.0, 'x2': 391.0, 'y1': 219.0, 'y2': 289.0, 'name': 'Watch'}, {'x1': 0.0, 'x2': 624.0, 'y1': 0.0, 'y2': 329.0, 'name': 'Bed'}, {'x1': 285.0, 'x2': 330.0, 'y1': 214.0, 'y2': 284.0, 'name': 'Watch'}]
Watch 0.51
hello
######
{'Watch': 'Bed', 'Bed': 'Watch'}
['Watch', 'Watch', 'Bed', 'Watch']


In [23]:
type(otput)

tuple

In [27]:
otput[-1]

['Watch', 'Watch', 'Bed', 'Watch']